# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-54b806aa8e-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    #sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Politics Workflow Series ("Presidents of countries") 

Consider the following exploratory information need:

> You investigating presidents of the republic, or similar roles, across international states around the world

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wdt:P39`     | position held   | predicate |
| `wd:Q248577`  | President of the republic | node      |
| `wd:Q11696`   | President of U.S.A.      | node      |
| `wd:Q332711`  | President of Italy        | node |
| `wd:Q38`      | Italy          | node |
| `wd:Q30`      | U.S.A.        | node |



Also consider

```
?p wdt:P39/wdt:P279* wd:Q248577  .  
```

is the BGP to retrieve all **presidents of the world countries through history**

## Workload Goals

1. Identify the BGP for obtaining important attributes for people, e.g., date of birth/death, gender, profession 

2. Identify the BGP to retrieve countries with  had at least once had a president

3. When was the first president of each country born? How many presidents each country had?

4. Is there a country that had at some point a woman as a president?

5. Analyze the number of presidents per country through time
 
   5.1 What are the top-5 countries for number of presidents? Which countries had the least?
   
   5.2 For how many presidents we know a date of death in each country?
   
   5.3 Which are the professions of different presidents? How many presidents had a specific profession?


In [3]:
q = f"""
select distinct ?name where{{
wd:Q11696 <http://schema.org/name> ?name
}}
"""
run_query(q)

[('name', 'President of the United States')]


1

In [4]:
#first of all, i want to find the identifier for the class "Person" or "Human" or the equivalent
#I'm going to try to find a president of the USA, then try and see which class(es) he belongs to

q = f"""
select distinct ?class ?name where{{
?person wdt:P39 wd:Q11696 ;
    wdt:P31 ?class .
    
?class <http://schema.org/name> ?name
}}
limit 100
"""
run_query(q)

[('class', 'http://www.wikidata.org/entity/Q1114461'), ('name', 'comics character')]
[('class', 'http://www.wikidata.org/entity/Q1307329'), ('name', 'extraterrestrials in fiction')]
[('class', 'http://www.wikidata.org/entity/Q18327510'), ('name', 'fictional robot')]
[('class', 'http://www.wikidata.org/entity/Q95074'), ('name', 'fictional character')]
[('class', 'http://www.wikidata.org/entity/Q15632617'), ('name', 'fictional human')]
[('class', 'http://www.wikidata.org/entity/Q15711870'), ('name', 'animated character')]
[('class', 'http://www.wikidata.org/entity/Q15773317'), ('name', 'television character')]
[('class', 'http://www.wikidata.org/entity/Q15773347'), ('name', 'film character')]
[('class', 'http://www.wikidata.org/entity/Q3658341'), ('name', 'literary character')]
[('class', 'http://www.wikidata.org/entity/Q5'), ('name', 'human')]
[('class', 'http://www.wikidata.org/entity/Q3716522'), ('name', 'Durlan')]


11

In [6]:
#let's retrieve properties for humans
q = f"""
select distinct ?p ?name where{{
?sub wdt:P31 wd:Q5 ;
    ?p ?object .
    
?p <http://schema.org/name> ?name
}}
limit 100
"""

run_query(q)

[('p', 'http://www.wikidata.org/prop/direct/P9271'), ('name', 'Kickstarter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2380'), ('name', 'French Sculpture Census artist ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3928'), ('name', 'MotoGP racer ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4421'), ('name', 'Sportbox.ru ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4015'), ('name', 'Vimeo identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P2985'), ('name', 'DBSE ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9053'), ('name', 'Projecto Vercial author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P10

100

In [8]:
#let's restrict the search of properties only to presidendts of the united states
q = f"""
select distinct ?p ?name where{{
?person wdt:P39 wd:Q11696 ;
    wdt:P31 wd:Q5;
    ?p ?object .
    
?p <http://schema.org/name> ?name .

filter regex(?name, '.*(death|birth|profession|gender|height|weight|child).*', 'i')
}}
limit 100
"""
run_query(q)

[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1477'), ('name', 'birth name')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P1971'), ('name', 'number of children')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death')]
[('p', 'http://www.wikidata.org/prop/direct/P2048'), ('name', 'height')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('name', 'sex or gender')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('name', 'child')]
[('p', 'http://www.wikidata.org/prop/direct/P509'), ('name', 'cause of death')]
[('p', 'http://www.wikidata.org/prop/direct/P569'), ('name', 'date of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P570'), ('name', 'date of death')]


11

In [10]:
#let's see if the various types of presidents have a superclass "President"
q = f"""
select distinct ?super ?name where{{
wd:Q11696 wdt:P279 ?super .
?super <http://schema.org/name> ?name
}}
"""
run_query(q)

[('super', 'http://www.wikidata.org/entity/Q2285706'), ('name', 'head of government')]
[('super', 'http://www.wikidata.org/entity/Q248577'), ('name', 'President of the Republic')]
[('super', 'http://www.wikidata.org/entity/Q48352'), ('name', 'head of state')]


3

In [11]:
#let's see if "President of the Republic" has a superclass as well
q = f"""
select distinct ?super ?name where{{
wd:Q248577 wdt:P279 ?super .
?super <http://schema.org/name> ?name
}}
"""
run_query(q)

[('super', 'http://www.wikidata.org/entity/Q30461'), ('name', 'president')]
[('super', 'http://www.wikidata.org/entity/Q48352'), ('name', 'head of state')]


2

In [13]:
#2. Identify the BGP to retrieve countries with  had at least once had a president

#let's find out if "President of the usa" has the usa as a country connected via the "country" property
#also let's do it for Italy

q = f"""
select distinct ?country ?name where{{
wd:Q11696 wdt:P17 ?country .
?country <http://schema.org/name> ?name
}}
"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]


1

In [14]:
q = f"""
select distinct ?country ?name where{{
wd:Q332711 wdt:P17 ?country .
?country <http://schema.org/name> ?name
}}
"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]


1

In [15]:
#finally let's see if the superclass "President" has countries connected to it
q = f"""
select distinct ?country ?name where{{
wd:Q30461 wdt:P17 ?country .
?country <http://schema.org/name> ?name
}}
"""
run_query(q)

Empty


0

In [16]:
#let's see if President of the Republic has countries connected to it
q = f"""
select distinct ?country ?name where{{
wd:Q248577 wdt:P17 ?country .
?country <http://schema.org/name> ?name
}}
"""
run_query(q)

Empty


0

In [17]:
#it makes sense that only the more specific subclasses "President of the USA" and "President of Italy" have a country
#let's query now all countries that had at least one president
#to do so we find Humans that held some position, using the * operator until we reach the superclass "President"

q = f"""
select distinct ?country ?name where{{
?person wdt:P31 wd:Q5 ;
    wdt:P39 ?position .

#assumption: President is only superclass and no person actually held/holds the charge
#of just "President", therefore using + should be correct
?position wdt:P279+ wd:Q30461 ;
    wdt:P17 ?country .

?country <http://schema.org/name> ?name
}}
order by asc(?name)
limit 50
"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q889'), ('name', 'Afghanistan')]
[('country', 'http://www.wikidata.org/entity/Q222'), ('name', 'Albania')]
[('country', 'http://www.wikidata.org/entity/Q262'), ('name', 'Algeria')]
[('country', 'http://www.wikidata.org/entity/Q916'), ('name', 'Angola')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('name', 'Argentina')]
[('country', 'http://www.wikidata.org/entity/Q399'), ('name', 'Armenia')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria')]
[('country', 'http://www.wikidata.org/entity/Q227'), ('name', 'Azerbaijan')]
[('country', 'http://www.wikidata.org/entity/Q902'), ('name', 'Bangladesh')]
[('country', 'http://www.wikidata.org/entity/Q184'), ('name', 'Belarus')]
[('country', 'http://www.wikidata.org/entity/Q962'), ('name', 'Benin')]
[('country', 'http://www.wikidata.org/entity/Q750'), ('name', 'Bolivia')]
[('country', 'http://www.wikidata.org/entity/Q225'), ('name', 'Bosnia and Herzegovina')]
[('country', 'h

50

In [18]:
#let's also return the name of the president in each country
q = f"""
select distinct ?country ?name ?positionName where{{
?person wdt:P31 wd:Q5 ;
    wdt:P39 ?position .

#assumption: no person helds directly the "President" charge
#therefore using the + s
?position wdt:P279+ wd:Q30461 ;
    <http://schema.org/name> ?positionName ;
    wdt:P17 ?country .

?country <http://schema.org/name> ?name
}}
order by asc(?name)
limit 50
"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q889'), ('name', 'Afghanistan'), ('positionName', 'President of Afghanistan')]
[('country', 'http://www.wikidata.org/entity/Q889'), ('name', 'Afghanistan'), ('positionName', 'Chairman of the Revolutionary Council of Afghanistan')]
[('country', 'http://www.wikidata.org/entity/Q222'), ('name', 'Albania'), ('positionName', 'President of Albania')]
[('country', 'http://www.wikidata.org/entity/Q262'), ('name', 'Algeria'), ('positionName', 'President of Algeria')]
[('country', 'http://www.wikidata.org/entity/Q916'), ('name', 'Angola'), ('positionName', 'President of Angola')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('name', 'Argentina'), ('positionName', 'President of Argentina')]
[('country', 'http://www.wikidata.org/entity/Q399'), ('name', 'Armenia'), ('positionName', 'President of Armenia')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria'), ('positionName', 'President of Austria')]
[('country', 'http://www.wi

50

In [20]:
#let's try and find countries that never had a president, if any
q = f"""
select distinct ?country ?name where{{
?person wdt:P31 wd:Q5 ;
    wdt:P39 ?position .
    
filter not exists{{
        ?position wdt:P279+ wd:Q30461 .
    }}
    
?potision wdt:P17 ?country .
?country <http://schema.org/name> ?name .
}}
limit 20
"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q1643555'), ('name', 'British Honduras')]
[('country', 'http://www.wikidata.org/entity/Q797214'), ('name', 'French Togoland')]
[('country', 'http://www.wikidata.org/entity/Q139315'), ('name', 'Dʿmt')]
[('country', 'http://www.wikidata.org/entity/Q491559'), ('name', 'First Republic of South Korea')]
[('country', 'http://www.wikidata.org/entity/Q10975458'), ('name', 'Maramureș')]
[('country', 'http://www.wikidata.org/entity/Q1649306'), ('name', 'Nyasaland')]
[('country', 'http://www.wikidata.org/entity/Q7435677'), ('name', 'Scotland in the Late Middle Ages')]
[('country', 'http://www.wikidata.org/entity/Q488521'), ('name', 'Fourth Republic of South Korea')]
[('country', 'http://www.wikidata.org/entity/Q10889269'), ('name', 'Aolai')]
[('country', 'http://www.wikidata.org/entity/Q2308237'), ('name', 'Ahom Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q756854'), ('name', 'Kosala')]
[('country', 'http://www.wikidata.org/entity/Q3825239')

20

In [21]:
#let's also display the name of the held position
q = f"""
select distinct ?country ?name ?positionName where{{
?person wdt:P31 wd:Q5 ;
    wdt:P39 ?position .
    
filter not exists{{
        ?position wdt:P279+ wd:Q30461 .
    }}
    
?potision wdt:P17 ?country ;
    <http://schema.org/name> ?positionName .
?country <http://schema.org/name> ?name .
}}
limit 20
"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q180573'), ('name', 'South Vietnam'), ('positionName', 'South Vietnam Air Force')]
[('country', 'http://www.wikidata.org/entity/Q2981019'), ('name', 'Ostikanate of Arminiya'), ('positionName', 'Arshamuni')]
[('country', 'http://www.wikidata.org/entity/Q10889269'), ('name', 'Aolai'), ('positionName', 'Aolai')]
[('country', 'http://www.wikidata.org/entity/Q10889269'), ('name', 'Aolai'), ('positionName', 'Mount Huaguo')]
[('country', 'http://www.wikidata.org/entity/Q6124144'), ('name', 'Jaipur State'), ('positionName', 'Jodhpur State')]
[('country', 'http://www.wikidata.org/entity/Q7646305'), ('name', 'Surinam'), ('positionName', 'Plaisance')]
[('country', 'http://www.wikidata.org/entity/Q449639'), ('name', 'Alodia'), ('positionName', 'Soba')]
[('country', 'http://www.wikidata.org/entity/Q156418'), ('name', 'Kingdom of Hawaiʻi'), ('positionName', 'Kingdom of Hawaiʻi')]
[('country', 'http://www.wikidata.org/entity/Q180573'), ('name', 'South Viet

20

In [23]:
#first of all let's see if "President" is subclass of something
q = f"""
select distinct ?super ?name where{{
wd:Q30461 wdt:P279 ?super .
?super <http://schema.org/name> ?name
}}
"""
run_query(q)

[('super', 'http://www.wikidata.org/entity/Q48352'), ('name', 'head of state')]
[('super', 'http://www.wikidata.org/entity/Q82955'), ('name', 'politician')]
[('super', 'http://www.wikidata.org/entity/Q294414'), ('name', 'public office')]


3

In [24]:
#now let's look at nodes that are subclass of "head of state"
headOfState = "wd:Q48352"
q = f"""
select distinct ?class ?name where{{
?class wdt:P279+ wd:Q48352 .
?class <http://schema.org/name> ?name
}}
limit 20
"""
run_query(q)

[('class', 'http://www.wikidata.org/entity/Q16138154'), ('name', 'ceremonial head of state')]
[('class', 'http://www.wikidata.org/entity/Q22076010'), ('name', 'duke of Mayenne')]
[('class', 'http://www.wikidata.org/entity/Q2922479'), ('name', 'Duke of Schomberg')]
[('class', 'http://www.wikidata.org/entity/Q3268015'), ('name', 'Duke of Montmorency')]
[('class', 'http://www.wikidata.org/entity/Q781203'), ('name', 'Duke of Berwick')]
[('class', 'http://www.wikidata.org/entity/Q61905410'), ('name', 'President of Puntland')]
[('class', 'http://www.wikidata.org/entity/Q729924'), ('name', 'Prince regent')]
[('class', 'http://www.wikidata.org/entity/Q21916441'), ('name', 'duke of Guise')]
[('class', 'http://www.wikidata.org/entity/Q22077580'), ('name', 'duke of Joyeuse')]
[('class', 'http://www.wikidata.org/entity/Q22939043'), ('name', 'king of Warnes')]
[('class', 'http://www.wikidata.org/entity/Q95147725'), ('name', 'Irish king')]
[('class', 'http://www.wikidata.org/entity/Q14946265'), ('na

20

In [25]:
#let's look for kings, dictators...
q = f"""
select distinct ?class ?name where{{
?class wdt:P279+ wd:Q48352 .
?class <http://schema.org/name> ?name .
filter regex(?name, '.*(king|dictator).*', 'i')
}}
limit 20
"""
run_query(q)

[('class', 'http://www.wikidata.org/entity/Q22939043'), ('name', 'king of Warnes')]
[('class', 'http://www.wikidata.org/entity/Q95147725'), ('name', 'Irish king')]
[('class', 'http://www.wikidata.org/entity/Q27213339'), ('name', 'King of Essex')]
[('class', 'http://www.wikidata.org/entity/Q3930923'), ('name', 'King of Sicily')]
[('class', 'http://www.wikidata.org/entity/Q14368159'), ('name', 'king of Armenia')]
[('class', 'http://www.wikidata.org/entity/Q41542585'), ('name', 'King of Lesotho')]
[('class', 'http://www.wikidata.org/entity/Q60296919'), ('name', 'king of Luang Prabang')]
[('class', 'http://www.wikidata.org/entity/Q58008017'), ('name', 'king of Galicia')]
[('class', 'http://www.wikidata.org/entity/Q65028901'), ('name', 'king of the Suebi')]
[('class', 'http://www.wikidata.org/entity/Q28166264'), ('name', 'King of Manipur')]
[('class', 'http://www.wikidata.org/entity/Q15725932'), ('name', 'king of Imereti')]
[('class', 'http://www.wikidata.org/entity/Q183318'), ('name', 'dic

20

In [33]:
Dictator = 'wd:Q183318'
#let's see countries with a dictator
q = f"""
select distinct ?personName ?name ?positionName where{{
?person wdt:P31 wd:Q5 .
?person wdt:P39 ?position .
?position wdt:P279+ wd:Q183318 .
?person <http://schema.org/name> ?personName .
?position wdt:P17 ?country .
?position <http://schema.org/name> ?positionName .
    
?country <http://schema.org/name> ?name .
}}
limit 20
"""
run_query(q)

[('personName', 'Hu Jintao'), ('name', "People's Republic of China"), ('positionName', 'General Secretary of the Chinese Communist Party')]
[('personName', 'Xi Jinping'), ('name', "People's Republic of China"), ('positionName', 'General Secretary of the Chinese Communist Party')]
[('personName', 'Juan Lindo y Zelaya'), ('name', 'Federal Republic of Central America'), ('positionName', 'Jefe Supremo del Estado de El Salvador')]
[('personName', 'Deng Xiaoping'), ('name', "People's Republic of China"), ('positionName', "paramount leader of the People's Republic of China")]
[('personName', 'Hu Yaobang'), ('name', "People's Republic of China"), ('positionName', 'General Secretary of the Chinese Communist Party')]
[('personName', 'Zhao Ziyang'), ('name', "People's Republic of China"), ('positionName', 'General Secretary of the Chinese Communist Party')]
[('personName', 'Juan Nepomuceno Barrundia Cepeda'), ('name', 'Federal Republic of Central America'), ('positionName', 'Jefe Supremo del Esta

20

In [34]:
#in cell 21 i made a spelling mistake.... "potision" instead of "position"
#let's correct the query

q = f"""
select distinct ?country ?name ?positionName where{{
?person wdt:P31 wd:Q5 ;
    wdt:P39 ?position .
    
filter not exists{{
        ?position wdt:P279+ wd:Q30461 .
    }}
    
?position wdt:P17 ?country ;
    <http://schema.org/name> ?positionName .
?country <http://schema.org/name> ?name .
}}
limit 20
"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q2670751'), ('name', 'Margraviate of Moravia'), ('positionName', 'Member of the Moravian Diet')]
[('country', 'http://www.wikidata.org/entity/Q139319'), ('name', 'Russian Republic'), ('positionName', 'Оrdinary professor')]
[('country', 'http://www.wikidata.org/entity/Q174306'), ('name', 'Republic of Genova'), ('positionName', 'doge of Genoa')]
[('country', 'http://www.wikidata.org/entity/Q1014'), ('name', 'Liberia'), ('positionName', 'Member of the House of Representatives of Liberia')]
[('country', 'http://www.wikidata.org/entity/Q1033'), ('name', 'Nigeria'), ('positionName', 'Chief Justice of Nigeria')]
[('country', 'http://www.wikidata.org/entity/Q12060881'), ('name', 'Imperial China'), ('positionName', 'Emperor of China')]
[('country', 'http://www.wikidata.org/entity/Q13426199'), ('name', 'Republic of China 1912–1949'), ('positionName', 'member of the Constituent National Assembly')]
[('country', 'http://www.wikidata.org/entity/Q13426199

20

In [35]:
#now the results make more sense. however filtering out non presidential positions still gives us a lot of noise
#we find mayors, members of various assemblies etc...

#earlier we found a class called "Head of State"
#let's use that too

HeadOfState = 'wd:Q48352'

q = f"""
select distinct ?country ?name ?positionName where{{
?person wdt:P31 wd:Q5 ;
    wdt:P39 ?position .
    
?position wdt:P279+ wd:Q48352 .
    
filter not exists{{
        ?position wdt:P279+ wd:Q30461 .
    }}
    
?position wdt:P17 ?country ;
    <http://schema.org/name> ?positionName .
?country <http://schema.org/name> ?name .
}}
limit 20
"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q777981'), ('name', 'Duchy of the Archipelago'), ('positionName', 'Duke of Naxos')]
[('country', 'http://www.wikidata.org/entity/Q208169'), ('name', 'Republic of Ragusa'), ('positionName', 'Rector')]
[('country', 'http://www.wikidata.org/entity/Q46652'), ('name', 'Malacca sultanate'), ('positionName', 'Sultan of Malacca')]
[('country', 'http://www.wikidata.org/entity/Q1013'), ('name', 'Lesotho'), ('positionName', 'King of Lesotho')]
[('country', 'http://www.wikidata.org/entity/Q12548'), ('name', 'Holy Roman Empire'), ('positionName', 'Fürst')]
[('country', 'http://www.wikidata.org/entity/Q129286'), ('name', 'British India'), ('positionName', 'Emperor of India')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('positionName', 'King of Scotland')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('positionName', 'Duke of Beaufort')]
[('country', 'http://www.wikidata.org/entity/Q

20

In [37]:
#much better. now for the sake of completeness, let's use the previous query and let's look if
#"General Secretary of the Chinese Communist Party" is returned as well

q = f"""
select distinct ?country ?name ?positionName where{{
?person wdt:P31 wd:Q5 ;
    wdt:P39 ?position .
    
?position wdt:P279+ wd:Q48352 .
    
filter not exists{{
        ?position wdt:P279+ wd:Q30461 .
    }}
    
?position wdt:P17 ?country ;
    <http://schema.org/name> ?positionName .
    
?country <http://schema.org/name> ?name .

filter (?positionName = 'General Secretary of the Chinese Communist Party') .
}}
limit 20
"""
run_query(q)

Empty


0

In [39]:
#let's remove "Head of State"
q = f"""
select distinct ?country ?name ?positionName where{{
?person wdt:P31 wd:Q5 ;
    wdt:P39 ?position .
        
filter not exists{{
        ?position wdt:P279+ wd:Q30461 .
    }}
    
?position wdt:P17 ?country ;
    <http://schema.org/name> ?positionName .
    
?country <http://schema.org/name> ?name .

filter (?positionName = 'General Secretary of the Chinese Communist Party') .
}}
limit 20
"""
run_query(q)

Empty


0

In [47]:
#maybe it's better to remove the filter against "President"

q = f"""
select distinct ?country ?name ?positionName where{{
?person wdt:P31 wd:Q5 ;
    wdt:P39 ?position .
    
?position wdt:P279+ wd:Q48352 .
    
?position wdt:P17 ?country ;
    <http://schema.org/name> ?positionName .
    
?country <http://schema.org/name> ?name .

filter regex (?positionName, '.*chinese communist party.*', 'i') .
}}
limit 20
"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q148'), ('name', "People's Republic of China"), ('positionName', 'General Secretary of the Chinese Communist Party')]


1

In [48]:
#turns out that using regex returns the expected result
#going back to the original query
q = f"""
select distinct ?country ?name ?positionName where{{
?person wdt:P31 wd:Q5 ;
    wdt:P39 ?position .
    
?position wdt:P279+ wd:Q48352 .
    
filter not exists{{
        ?position wdt:P279+ wd:Q30461 .
    }}
    
?position wdt:P17 ?country ;
    <http://schema.org/name> ?positionName .
    
?country <http://schema.org/name> ?name .

filter regex (?positionName, '.*chinese communist party.*', 'i') .
}}
limit 20
"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q148'), ('name', "People's Republic of China"), ('positionName', 'General Secretary of the Chinese Communist Party')]


1

In [51]:
#date of birth of first president in each country

#this is the 5th notebook i complete so i already know that retrieving the position via the
#"position held" property we have available is probably not going to give me information about the order in which
#each president was elected
#knowing this we simply get the first president to be born in each country
q = f"""
select ?countryName (min(?dateOfBirth) as ?dob) where{{
?president wdt:P31 wd:Q5 ;
    <http://schema.org/name> ?presidentName ;
    wdt:P569 ?dateOfBirth ;
    wdt:P39 ?position .
    
?position wdt:P279+ wd:Q30461 .
?position wdt:P17 ?country .
?country <http://schema.org/name> ?countryName
}}
order by asc(?countryName)
limit 50
"""
run_query(q)

[('countryName', 'Afghanistan'), ('dob', '1909-07-18T00:00:00Z')]
[('countryName', 'Albania'), ('dob', '1844-10-16T00:00:00Z')]
[('countryName', 'Algeria'), ('dob', '1899-08-24T00:00:00Z')]
[('countryName', 'Angola'), ('dob', '1922-09-17T00:00:00Z')]
[('countryName', 'Argentina'), ('dob', '1757-06-18T00:00:00Z')]
[('countryName', 'Armenia'), ('dob', '1945-01-09T00:00:00Z')]
[('countryName', 'Austria'), ('dob', '1858-08-15T00:00:00Z')]
[('countryName', 'Azerbaijan'), ('dob', '1888-07-26T00:00:00Z')]
[('countryName', 'Bangladesh'), ('dob', '1906-01-01T00:00:00Z')]
[('countryName', 'Belarus'), ('dob', '1954-08-30T00:00:00Z')]
[('countryName', 'Benin'), ('dob', '1913-01-01T00:00:00Z')]
[('countryName', 'Bolivia'), ('dob', '1751-07-26T00:00:00Z')]
[('countryName', 'Bosnia and Herzegovina'), ('dob', '1930-07-07T00:00:00Z')]
[('countryName', 'Botswana'), ('dob', '1921-07-01T00:00:00Z')]
[('countryName', 'Brazil'), ('dob', '1827-08-05T00:00:00Z')]
[('countryName', 'Burkina Faso'), ('dob', '191

50

In [53]:
#that query returns the country and the minimum date of birth for a president
#let's return also the president

subQ = f"""
select ?country (min(?dateOfBirth) as ?minDob) where{{
?president wdt:P31 wd:Q5 ;
    wdt:P569 ?dateOfBirth ;
    wdt:P39 ?position .
    
?position wdt:P279+ wd:Q30461 .
?position wdt:P17 ?country .
}}
"""

q = f"""
select distinct ?countryName ?name ?dateOfBirth where{{
?president wdt:P31 wd:Q5 ;
    wdt:P569 ?dateOfBirth ;
    <http://schema.org/name> ?name ;
    wdt:P39 ?position .
    
?position wdt:P279+ wd:Q30461 .
?position wdt:P17 ?c .

    {{
        select ?country (min(?dateOfBirth) as ?minDob) where{{

?president wdt:P31 wd:Q5 ;

    wdt:P569 ?dateOfBirth ;

    wdt:P39 ?position .

    

?position wdt:P279+ wd:Q30461 .

?position wdt:P17 ?country .

}}



    }}
    
?c <http://schema.org/name> ?countryName
    
filter(?dateOfBirth = ?minDob && ?c = ?country)

}}
order by asc(?countryName)
limit 50
"""

run_query(q)

[('countryName', 'Afghanistan'), ('name', 'Mohammed Daoud Khan'), ('dateOfBirth', '1909-07-18T00:00:00Z')]
[('countryName', 'Albania'), ('name', 'Ismail Qemali'), ('dateOfBirth', '1844-10-16T00:00:00Z')]
[('countryName', 'Algeria'), ('name', 'Ferhat Abbas'), ('dateOfBirth', '1899-08-24T00:00:00Z')]
[('countryName', 'Angola'), ('name', 'Agostinho Neto'), ('dateOfBirth', '1922-09-17T00:00:00Z')]
[('countryName', 'Argentina'), ('name', 'Gervasio Antonio de Posadas'), ('dateOfBirth', '1757-06-18T00:00:00Z')]
[('countryName', 'Armenia'), ('name', 'Levon Ter-Petrosyan'), ('dateOfBirth', '1945-01-09T00:00:00Z')]
[('countryName', 'Austria'), ('name', 'Michael Hainisch'), ('dateOfBirth', '1858-08-15T00:00:00Z')]
[('countryName', 'Azerbaijan'), ('name', 'Gazanfar Musabekov'), ('dateOfBirth', '1888-07-26T00:00:00Z')]
[('countryName', 'Bangladesh'), ('name', 'Abdus Sattar'), ('dateOfBirth', '1906-01-01T00:00:00Z')]
[('countryName', 'Belarus'), ('name', 'Alexander Lukashenko'), ('dateOfBirth', '195

50

In [54]:
#let's look for italy
subQ = f"""
select ?country (min(?dateOfBirth) as ?minDob) where{{
?president wdt:P31 wd:Q5 ;
    wdt:P569 ?dateOfBirth ;
    wdt:P39 ?position .
    
?position wdt:P279+ wd:Q30461 .
?position wdt:P17 ?country .
}}
"""

q = f"""
select distinct ?countryName ?name ?dateOfBirth where{{
?president wdt:P31 wd:Q5 ;
    wdt:P569 ?dateOfBirth ;
    <http://schema.org/name> ?name ;
    wdt:P39 ?position .
    
?position wdt:P279+ wd:Q30461 .
?position wdt:P17 ?c .

    {{
        select ?country (min(?dateOfBirth) as ?minDob) where{{

?president wdt:P31 wd:Q5 ;

    wdt:P569 ?dateOfBirth ;

    wdt:P39 ?position .

    

?position wdt:P279+ wd:Q30461 .

?position wdt:P17 ?country .

}}



    }}
    
?c <http://schema.org/name> ?countryName
    
filter(?dateOfBirth = ?minDob && ?c = ?country).
filter regex(?countryName, 'Italy')

}}
order by asc(?countryName)
limit 50
"""

run_query(q)

[('countryName', 'Italy'), ('name', 'Niccolò Nobili'), ('dateOfBirth', '1830-12-15T00:00:00Z')]
[('countryName', 'Kingdom of Italy'), ('name', 'Niccolò Nobili'), ('dateOfBirth', '1830-12-15T00:00:00Z')]


2

In [56]:
#can't find much for this person on google. Let's try with "Presidente della Repubblica Italiana"
subQ = f"""
select ?country (min(?dateOfBirth) as ?minDob) where{{
?president wdt:P31 wd:Q5 ;
    wdt:P569 ?dateOfBirth ;
    wdt:P39 ?position .
    
?position wdt:P17 ?country ;
    wdt:P31 wd:Q332711
}}
"""

q = f"""
select distinct ?countryName ?name ?dateOfBirth where{{
?president wdt:P31 wd:Q5 ;
    wdt:P569 ?dateOfBirth ;
    <http://schema.org/name> ?name ;
    wdt:P39 ?position .
    
?position wdt:P17 ?c ;
    wdt:P31 wd:Q332711

    {{
        select ?country (min(?dateOfBirth) as ?minDob) where{{

?president wdt:P31 wd:Q5 ;

    wdt:P569 ?dateOfBirth ;

    wdt:P39 ?position .

    

?position wdt:P17 ?country ;

    wdt:P31 wd:Q332711

}}



    }}
    
?c <http://schema.org/name> ?countryName
    
filter(?dateOfBirth = ?minDob && ?c = ?country).
filter regex(?countryName, 'Italy')

}}
order by asc(?countryName)
limit 50
"""

run_query(q)

Empty


0

In [60]:
#let's use a simpler query
q = f"""
select distinct ?personName ?dateOfBirth where{{
?president wdt:P31 wd:Q5 ;
    wdt:P569 ?dateOfBirth ;
    <http://schema.org/name> ?personName ;
    wdt:P39 wd:Q332711 .

}}
order by asc(?dateOfBirth)
limit 50
"""
run_query(q)

[('personName', 'Luigi Einaudi'), ('dateOfBirth', '1874-03-24T00:00:00Z')]
[('personName', 'Enrico de Nicola'), ('dateOfBirth', '1877-11-09T00:00:00Z')]
[('personName', 'Giovanni Gronchi'), ('dateOfBirth', '1887-09-10T00:00:00Z')]
[('personName', 'Antonio Segni'), ('dateOfBirth', '1891-02-02T00:00:00Z')]
[('personName', 'Sandro Pertini'), ('dateOfBirth', '1896-09-25T00:00:00Z')]
[('personName', 'Giuseppe Saragat'), ('dateOfBirth', '1898-09-19T00:00:00Z')]
[('personName', 'Giovanni Leone'), ('dateOfBirth', '1908-11-03T00:00:00Z')]
[('personName', 'Oscar Luigi Scalfaro'), ('dateOfBirth', '1918-09-09T00:00:00Z')]
[('personName', 'Carlo Azeglio Ciampi'), ('dateOfBirth', '1920-12-09T00:00:00Z')]
[('personName', 'Giorgio Napolitano'), ('dateOfBirth', '1925-06-29T00:00:00Z')]
[('personName', 'Francesco Cossiga'), ('dateOfBirth', '1928-07-26T00:00:00Z')]
[('personName', 'Sergio Mattarella'), ('dateOfBirth', '1941-07-23T00:00:00Z')]


12

In [61]:
#Query in cell 56 was wrong, let's correct it
subQ = f"""
select ?country (min(?dateOfBirth) as ?minDob) where{{
?president wdt:P31 wd:Q5 ;
    wdt:P569 ?dateOfBirth ;
    wdt:P39 ?position .
    
?position wdt:P17 ?country ;
    wdt:P279 wd:Q248577
}}
"""

q = f"""
select distinct ?countryName ?name ?dateOfBirth where{{
?president wdt:P31 wd:Q5 ;
    wdt:P569 ?dateOfBirth ;
    <http://schema.org/name> ?name ;
    wdt:P39 ?position .
    
?position wdt:P17 ?c ;
    wdt:P279 wd:Q248577

    {{
        select ?country (min(?dateOfBirth) as ?minDob) where{{

?president wdt:P31 wd:Q5 ;

    wdt:P569 ?dateOfBirth ;

    wdt:P39 ?position .

    

?position wdt:P17 ?country ;

    wdt:P279 wd:Q248577

}}



    }}
    
?c <http://schema.org/name> ?countryName
    
filter(?dateOfBirth = ?minDob && ?c = ?country).
filter regex(?countryName, 'Italy')

}}
order by asc(?countryName)
limit 50
"""

run_query(q)

[('countryName', 'Italy'), ('name', 'Luigi Einaudi'), ('dateOfBirth', '1874-03-24T00:00:00Z')]


1

In [62]:
#and of course the second president was born before the first..
#maybe a country has a property to the first president?

q = f"""
select ?p ?name where{{
wd:Q38 ?p ?obj .
?p <http://schema.org/name> ?name .
filter regex(?name, '.*first.*', 'i') .
}}
"""
run_query(q)

[('p', 'http://www.wikidata.org/prop/direct/P6766'), ('name', "Who's on First ID")]


1

In [66]:
#maybe not ordering the results will give us the original order
q = f"""
select ?personName ?dateOfBirth where{{
?person wdt:P31 wd:Q5 ;
    wdt:P39 wd:Q332711 ;
    wdt:P569 ?dateOfBirth ;
    <http://schema.org/name> ?personName
}}
"""
run_query(q)

[('personName', 'Giorgio Napolitano'), ('dateOfBirth', '1925-06-29T00:00:00Z')]
[('personName', 'Carlo Azeglio Ciampi'), ('dateOfBirth', '1920-12-09T00:00:00Z')]
[('personName', 'Oscar Luigi Scalfaro'), ('dateOfBirth', '1918-09-09T00:00:00Z')]
[('personName', 'Giovanni Gronchi'), ('dateOfBirth', '1887-09-10T00:00:00Z')]
[('personName', 'Francesco Cossiga'), ('dateOfBirth', '1928-07-26T00:00:00Z')]
[('personName', 'Giuseppe Saragat'), ('dateOfBirth', '1898-09-19T00:00:00Z')]
[('personName', 'Enrico de Nicola'), ('dateOfBirth', '1877-11-09T00:00:00Z')]
[('personName', 'Sandro Pertini'), ('dateOfBirth', '1896-09-25T00:00:00Z')]
[('personName', 'Sergio Mattarella'), ('dateOfBirth', '1941-07-23T00:00:00Z')]
[('personName', 'Luigi Einaudi'), ('dateOfBirth', '1874-03-24T00:00:00Z')]
[('personName', 'Giovanni Leone'), ('dateOfBirth', '1908-11-03T00:00:00Z')]
[('personName', 'Antonio Segni'), ('dateOfBirth', '1891-02-02T00:00:00Z')]


12

In [69]:
#is "President of Republic" superclass actually directly used as "Position held" object?
q = f"""
select distinct ?president ?name where{{
?president wdt:P31 wd:Q5 ;
    <http://schema.org/name> ?name ;
    wdt:P39 wd:Q248577 .

}}
limit 50
"""
run_query(q)

[('president', 'http://www.wikidata.org/entity/Q561303'), ('name', 'Nicolas Grunitzky')]
[('president', 'http://www.wikidata.org/entity/Q99522144'), ('name', 'Bah Ndaw')]


2

In [71]:
#do those 2 have other objects regarding the position held?
q = f"""
select distinct ?name ?position ?positionName where{{
?president wdt:P31 wd:Q5 ;
    <http://schema.org/name> ?name ;
    wdt:P39 ?position.
    
?position <http://schema.org/name> ?positionName .

filter(?president = wd:Q561303 || ?president = wd:Q99522144).
}}
order by ?name
"""
run_query(q)

[('name', 'Bah Ndaw'), ('position', 'http://www.wikidata.org/entity/Q1162163'), ('positionName', 'director')]
[('name', 'Bah Ndaw'), ('position', 'http://www.wikidata.org/entity/Q248577'), ('positionName', 'President of the Republic')]
[('name', 'Bah Ndaw'), ('position', 'http://www.wikidata.org/entity/Q2518691'), ('positionName', 'defence minister')]
[('name', 'Bah Ndaw'), ('position', 'http://www.wikidata.org/entity/Q369894'), ('positionName', 'aide-de-camp')]
[('name', 'Nicolas Grunitzky'), ('position', 'http://www.wikidata.org/entity/Q248577'), ('positionName', 'President of the Republic')]
[('name', 'Nicolas Grunitzky'), ('position', 'http://www.wikidata.org/entity/Q28015471'), ('positionName', 'President of Togo')]
[('name', 'Nicolas Grunitzky'), ('position', 'http://www.wikidata.org/entity/Q3044918'), ('positionName', 'member of the French National Assembly')]
[('name', 'Nicolas Grunitzky'), ('position', 'http://www.wikidata.org/entity/Q30100661'), ('positionName', 'Prime Minist

8

In [72]:
#there is 1 person that has position held "President of the Republic" without having some other more specific presidency
#therefore i shouldn't use the query ?position wdt:P279+ wd:Q248577 since it would ignore that one person
#however i still need to bind the variable ?position to retrieve the country relative to that specific presidency
#an alternative would be to retrieve the country of the person and not of the presidency, but it may return wrong results
#it's better to count one person less than to have more imprecise results

#3.2 How many presidents did each country have?
q = f"""
select ?country ?countryName (count(?president) as ?nPres) where{{
?president wdt:P31 wd:Q5 ;
    wdt:P39 ?position .
    
?position wdt:P279+ wd:Q248577 ;
    wdt:P17 ?country .
    
?country <http://schema.org/name> ?countryName .

}}
order by desc (?nPres)
limit 50
"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q783'), ('countryName', 'Honduras'), ('nPres', '83')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America'), ('nPres', '78')]
[('country', 'http://www.wikidata.org/entity/Q739'), ('countryName', 'Colombia'), ('nPres', '60')]
[('country', 'http://www.wikidata.org/entity/Q96'), ('countryName', 'Mexico'), ('nPres', '59')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina'), ('nPres', '59')]
[('country', 'http://www.wikidata.org/entity/Q717'), ('countryName', 'Venezuela'), ('nPres', '58')]
[('country', 'http://www.wikidata.org/entity/Q733'), ('countryName', 'Paraguay'), ('nPres', '57')]
[('country', 'http://www.wikidata.org/entity/Q790'), ('countryName', 'Haiti'), ('nPres', '55')]
[('country', 'http://www.wikidata.org/entity/Q419'), ('countryName', 'Peru'), ('nPres', '53')]
[('country', 'http://www.wikidata.org/entity/Q774'), ('countryName', 'Guatemala'), ('nPres', '53')]
[(

50

In [73]:
#there is something wrong. usa has 46 presidents...
#is it counting re-elections?
q = f"""
select ?country ?countryName (count(distinct ?president) as ?nPres) where{{
?president wdt:P31 wd:Q5 ;
    wdt:P39 ?position .
    
?position wdt:P279+ wd:Q248577 ;
    wdt:P17 ?country .
    
?country <http://schema.org/name> ?countryName .

}}
order by desc (?nPres)
limit 50
"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q783'), ('countryName', 'Honduras'), ('nPres', '83')]
[('country', 'http://www.wikidata.org/entity/Q739'), ('countryName', 'Colombia'), ('nPres', '60')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina'), ('nPres', '59')]
[('country', 'http://www.wikidata.org/entity/Q96'), ('countryName', 'Mexico'), ('nPres', '59')]
[('country', 'http://www.wikidata.org/entity/Q717'), ('countryName', 'Venezuela'), ('nPres', '58')]
[('country', 'http://www.wikidata.org/entity/Q733'), ('countryName', 'Paraguay'), ('nPres', '57')]
[('country', 'http://www.wikidata.org/entity/Q790'), ('countryName', 'Haiti'), ('nPres', '55')]
[('country', 'http://www.wikidata.org/entity/Q774'), ('countryName', 'Guatemala'), ('nPres', '53')]
[('country', 'http://www.wikidata.org/entity/Q419'), ('countryName', 'Peru'), ('nPres', '53')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America'), ('nPres', '45')]
[(

50

In [74]:
#now usa has 45, the internet says 46. almost perfect. Is joe biden not in this dataset yet?
USA = 'wd:Q30'
q = f"""
select ?presName where{{
?president wdt:P31 wd:Q5 ;
    wdt:P39 ?position ;
    <http://schema.org/name> ?presName .
    
?position wdt:P279+ wd:Q248577 ;
    wdt:P17 wd:Q30 .
    
filter regex(?presName, '.*biden.*', 'i').

}}
order by desc (?nPres)
limit 50
"""
run_query(q)

[('presName', 'Joe Biden')]


1

In [76]:
#let's list them all and check manually
q = f"""
select distinct ?presName where{{
?president wdt:P31 wd:Q5 ;
    wdt:P39 ?position ;
    <http://schema.org/name> ?presName .
    
?position wdt:P279+ wd:Q248577 ;
    wdt:P17 wd:Q30 .
    
}}
order by ?presName
limit 50
"""
run_query(q)

[('presName', 'Abraham Lincoln')]
[('presName', 'Andrew Jackson')]
[('presName', 'Andrew Johnson')]
[('presName', 'Barack Obama')]
[('presName', 'Benjamin Harrison')]
[('presName', 'Bill Clinton')]
[('presName', 'Calvin Coolidge')]
[('presName', 'Chester A. Arthur')]
[('presName', 'Donald Trump')]
[('presName', 'Dwight D. Eisenhower')]
[('presName', 'Franklin Delano Roosevelt')]
[('presName', 'Franklin Pierce')]
[('presName', 'George H. W. Bush')]
[('presName', 'George W. Bush')]
[('presName', 'George Washington')]
[('presName', 'Gerald Ford')]
[('presName', 'Grover Cleveland')]
[('presName', 'Harry S Truman')]
[('presName', 'Herbert Hoover')]
[('presName', 'James A. Garfield')]
[('presName', 'James Buchanan')]
[('presName', 'James K. Polk')]
[('presName', 'James Madison')]
[('presName', 'James Monroe')]
[('presName', 'Jimmy Carter')]
[('presName', 'Joe Biden')]
[('presName', 'John Adams')]
[('presName', 'John F. Kennedy')]
[('presName', 'John Quincy Adams')]
[('presName', 'John Tyler'

45

In [78]:
#4 is there a country with a woman president at some point in history?

#let's do it in a roundabout way
#list all genders of every president of the republic

q = f"""
select distinct ?genderName where{{
?person wdt:P31 wd:Q5 ;
    wdt:P39/wdt:P279* wd:Q248577 .
    
?person wdt:P21 ?gender .
?gender <http://schema.org/name> ?genderName .
}}
"""
run_query(q)

[('genderName', 'female')]
[('genderName', 'male')]


2

In [81]:
#let's now list those countries
q = f"""
select distinct ?countryName where{{
?person wdt:P31 wd:Q5 ;
    wdt:P21 ?gender ;
    wdt:P39 ?position .
    
?position wdt:P279+ wd:Q248577 ;
    wdt:P17 ?country .
    
?country <http://schema.org/name> ?countryName .

?gender <http://schema.org/name> ?genderName .
filter regex(?genderName, 'female') .
}}
order by asc(?countryName)
limit 50
"""
run_query(q)

[('countryName', 'Argentina')]
[('countryName', 'Bosnia and Herzegovina')]
[('countryName', 'Brazil')]
[('countryName', 'Burundi')]
[('countryName', 'Central African Republic')]
[('countryName', 'Costa Rica')]
[('countryName', 'Croatia')]
[('countryName', 'Estonia')]
[('countryName', 'Ethiopia')]
[('countryName', 'Georgia')]
[('countryName', 'Greece')]
[('countryName', 'Guyana')]
[('countryName', 'Haiti')]
[('countryName', 'Iceland')]
[('countryName', 'India')]
[('countryName', 'Indonesia')]
[('countryName', 'Kosovo')]
[('countryName', 'Kyrgyzstan')]
[('countryName', 'Latvia')]
[('countryName', 'Lithuania')]
[('countryName', 'Marshall Islands')]
[('countryName', 'Mauritius')]
[('countryName', 'Moldova')]
[('countryName', 'Mongolia')]
[('countryName', 'Nepal')]
[('countryName', 'Philippines')]
[('countryName', 'Poland')]
[('countryName', 'Serbia')]
[('countryName', 'Singapore')]
[('countryName', 'South Africa')]
[('countryName', 'South Korea')]
[('countryName', 'Sri Lanka')]
[('countryN

36

In [82]:
#let's see the name of those people
q = f"""
select distinct ?countryName ?name where{{
?person wdt:P31 wd:Q5 ;
    wdt:P21 ?gender ;
    <http://schema.org/name> ?name ;
    wdt:P39 ?position .
    
?position wdt:P279+ wd:Q248577 ;
    wdt:P17 ?country .
    
?country <http://schema.org/name> ?countryName .

?gender <http://schema.org/name> ?genderName .
filter regex(?genderName, 'female') .
}}
order by asc(?countryName)
limit 50
"""
run_query(q)

[('countryName', 'Argentina'), ('name', 'Cristina Fernández de Kirchner')]
[('countryName', 'Argentina'), ('name', 'Isabel Martínez de Perón')]
[('countryName', 'Bosnia and Herzegovina'), ('name', 'Biljana Plavšić')]
[('countryName', 'Bosnia and Herzegovina'), ('name', 'Željka Cvijanović')]
[('countryName', 'Bosnia and Herzegovina'), ('name', 'Srebrenka Golić')]
[('countryName', 'Brazil'), ('name', 'Dilma Rousseff')]
[('countryName', 'Burundi'), ('name', 'Sylvie Kinigi')]
[('countryName', 'Central African Republic'), ('name', 'Catherine Samba-Panza')]
[('countryName', 'Costa Rica'), ('name', 'Laura Chinchilla')]
[('countryName', 'Croatia'), ('name', 'Ema Derossi-Bjelajac')]
[('countryName', 'Croatia'), ('name', 'Kolinda Grabar-Kitarović')]
[('countryName', 'Estonia'), ('name', 'Kersti Kaljulaid')]
[('countryName', 'Ethiopia'), ('name', 'Sahle-Work Zewde')]
[('countryName', 'Georgia'), ('name', 'Salome Zurabishvili')]
[('countryName', 'Georgia'), ('name', 'Nino Burjanadze')]
[('countryN

45

In [83]:
#point 5.1 was answered in a previous query
#let's re-write that
q = f"""
select ?country ?countryName (count(distinct ?president) as ?nPres) where{{
?president wdt:P31 wd:Q5 ;
    wdt:P39 ?position .
    
?position wdt:P279+ wd:Q248577 ;
    wdt:P17 ?country .
    
?country <http://schema.org/name> ?countryName .

}}
order by desc (?nPres)
limit 5
"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q783'), ('countryName', 'Honduras'), ('nPres', '83')]
[('country', 'http://www.wikidata.org/entity/Q739'), ('countryName', 'Colombia'), ('nPres', '60')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina'), ('nPres', '59')]
[('country', 'http://www.wikidata.org/entity/Q96'), ('countryName', 'Mexico'), ('nPres', '59')]
[('country', 'http://www.wikidata.org/entity/Q717'), ('countryName', 'Venezuela'), ('nPres', '58')]


5

In [84]:
#to answer point 5.2: for how many presidents do we know the date of death in each country?
#we need to use a subquery to select each president only once

subQ = f"""
select distinct ?country ?president ?dateOfDeath where{{
?president wdt:P31 wd:Q5 ;
    wdt:P570 ?dateOfDeath ;
    wdt:P39 ?position .
    
?position wdt:P279+ wd:Q248577 ;
    wdt:P17 ?country .
}}
"""

q = f"""
select ?country ?countryName (count(?dateOfDeath) as ?howmany) where{{
?country <http://schema.org/name> ?countryName .
    {{
        select distinct ?country ?president ?dateOfDeath where{{

?president wdt:P31 wd:Q5 ;

    wdt:P570 ?dateOfDeath ;

    wdt:P39 ?position .

    

?position wdt:P279+ wd:Q248577 ;

    wdt:P17 ?country .

}}



    }}
}}
group by ?country ?countryName
order by desc(?howmany)
limit 50
"""
run_query(q)

[('country', 'http://www.wikidata.org/entity/Q783'), ('countryName', 'Honduras'), ('howmany', '73')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('countryName', 'Argentina'), ('howmany', '60')]
[('country', 'http://www.wikidata.org/entity/Q717'), ('countryName', 'Venezuela'), ('howmany', '58')]
[('country', 'http://www.wikidata.org/entity/Q739'), ('countryName', 'Colombia'), ('howmany', '58')]
[('country', 'http://www.wikidata.org/entity/Q96'), ('countryName', 'Mexico'), ('howmany', '52')]
[('country', 'http://www.wikidata.org/entity/Q733'), ('countryName', 'Paraguay'), ('howmany', '51')]
[('country', 'http://www.wikidata.org/entity/Q790'), ('countryName', 'Haiti'), ('howmany', '50')]
[('country', 'http://www.wikidata.org/entity/Q774'), ('countryName', 'Guatemala'), ('howmany', '45')]
[('country', 'http://www.wikidata.org/entity/Q419'), ('countryName', 'Peru'), ('howmany', '44')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America'), (

50

In [85]:
#we still don't have the property related to a profession
#let's try and find it using italian presidents

q = f"""
select distinct ?property ?name where{{
?president wdt:P31 wd:Q5 ;
    wdt:P39 wd:Q332711 ;
    ?property ?o .

?property <http://schema.org/name> ?name
}}
"""
run_query(q)

[('property', 'http://www.wikidata.org/prop/direct/P9871'), ('name', 'INAPP author ID')]
[('property', 'http://www.wikidata.org/prop/direct/P9791'), ('name', 'ASE person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P5664'), ('name', 'Savoirs ENS ID')]
[('property', 'http://www.wikidata.org/prop/direct/P8648'), ('name', 'Ministry of the Interior of Italy ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('property', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('property', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('property', 'http://www.wikidata.or

186

In [86]:
occupation = 'wdt:P106'

#let's see all professions of italian presidents
q = f"""
select distinct ?name ?profession ?professionName where{{
?president wdt:P31 wd:Q5 ;
    <http://schema.org/name> ?name ;
    wdt:P39 wd:Q332711 ;
    wdt:P106 ?profession .

?profession <http://schema.org/name> ?professionName 
}}
"""
run_query(q)

[('name', 'Giorgio Napolitano'), ('profession', 'http://www.wikidata.org/entity/Q82955'), ('professionName', 'politician')]
[('name', 'Carlo Azeglio Ciampi'), ('profession', 'http://www.wikidata.org/entity/Q82955'), ('professionName', 'politician')]
[('name', 'Carlo Azeglio Ciampi'), ('profession', 'http://www.wikidata.org/entity/Q188094'), ('professionName', 'economist')]
[('name', 'Carlo Azeglio Ciampi'), ('profession', 'http://www.wikidata.org/entity/Q806798'), ('professionName', 'banker')]
[('name', 'Oscar Luigi Scalfaro'), ('profession', 'http://www.wikidata.org/entity/Q82955'), ('professionName', 'politician')]
[('name', 'Oscar Luigi Scalfaro'), ('profession', 'http://www.wikidata.org/entity/Q40348'), ('professionName', 'lawyer')]
[('name', 'Giovanni Gronchi'), ('profession', 'http://www.wikidata.org/entity/Q82955'), ('professionName', 'politician')]
[('name', 'Francesco Cossiga'), ('profession', 'http://www.wikidata.org/entity/Q82955'), ('professionName', 'politician')]
[('name'

30

In [87]:
#5.3 Which are the professions of different presidents? How many presidents had a specific profession?
#the first part of 5.3 can be answered with the previous query with some minor modifications
q = f"""
select distinct ?name ?profession ?professionName where{{
?president wdt:P31 wd:Q5 ;
    <http://schema.org/name> ?name ;
    wdt:P39/wdt:P279* wd:Q248577 ;
    wdt:P106 ?profession .

?profession <http://schema.org/name> ?professionName 
}}
limit 50
"""
run_query(q)

[('name', 'Francisco Sagasti'), ('profession', 'http://www.wikidata.org/entity/Q2267418'), ('professionName', 'industrial engineer')]
[('name', 'Yahya Jammeh'), ('profession', 'http://www.wikidata.org/entity/Q26267537'), ('professionName', 'serial rapist')]
[('name', 'Bashar al-Assad'), ('profession', 'http://www.wikidata.org/entity/Q12013238'), ('professionName', 'ophthalmologist')]
[('name', 'Kwame Nkrumah'), ('profession', 'http://www.wikidata.org/entity/Q1569495'), ('professionName', 'lecturer')]
[('name', 'Yoweri Museveni'), ('profession', 'http://www.wikidata.org/entity/Q183318'), ('professionName', 'dictator')]
[('name', 'Georgios Papadopoulos'), ('profession', 'http://www.wikidata.org/entity/Q183318'), ('professionName', 'dictator')]
[('name', 'Bamir Topi'), ('profession', 'http://www.wikidata.org/entity/Q202883'), ('professionName', 'veterinarian')]
[('name', 'S. R. Nathan'), ('profession', 'http://www.wikidata.org/entity/Q212238'), ('professionName', 'civil servant')]
[('name

50

In [88]:
#How many presidents had a specific profession?
q = f"""
select ?profession ?professionName (count(?president) as ?howmany) where{{
?president wdt:P31 wd:Q5 ;
    wdt:P39/wdt:P279* wd:Q248577 ;
    wdt:P106 ?profession .

?profession <http://schema.org/name> ?professionName 
}}
group by ?profession ?professionName
order by desc(?howmany)
limit 50
"""
run_query(q)

[('profession', 'http://www.wikidata.org/entity/Q82955'), ('professionName', 'politician'), ('howmany', '1583')]
[('profession', 'http://www.wikidata.org/entity/Q40348'), ('professionName', 'lawyer'), ('howmany', '360')]
[('profession', 'http://www.wikidata.org/entity/Q193391'), ('professionName', 'diplomat'), ('howmany', '212')]
[('profession', 'http://www.wikidata.org/entity/Q47064'), ('professionName', 'military personnel'), ('howmany', '200')]
[('profession', 'http://www.wikidata.org/entity/Q372436'), ('professionName', 'statesperson'), ('howmany', '91')]
[('profession', 'http://www.wikidata.org/entity/Q1930187'), ('professionName', 'journalist'), ('howmany', '83')]
[('profession', 'http://www.wikidata.org/entity/Q36180'), ('professionName', 'writer'), ('howmany', '80')]
[('profession', 'http://www.wikidata.org/entity/Q188094'), ('professionName', 'economist'), ('howmany', '75')]
[('profession', 'http://www.wikidata.org/entity/Q1622272'), ('professionName', 'university teacher'), (

50